# Deep Learning
Assignment 2

Previously in 1_notmnist.ipynb, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.


In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in 1_notmnist.ipynb.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:

    - data as a flat matrix,
    - labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


## We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:

    First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

    with graph.as_default():
        ...

    Then you can run the operations on this graph as many times as you want by calling session.run(), providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

    with tf.Session(graph=graph) as session:
        ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:


In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



Let's run this computation and iterate:

In [5]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 18.008060
Training accuracy: 4.9%
Validation accuracy: 7.9%
Loss at step 100: 2.333064
Training accuracy: 72.7%
Validation accuracy: 71.0%
Loss at step 200: 1.883035
Training accuracy: 75.3%
Validation accuracy: 73.3%
Loss at step 300: 1.630958
Training accuracy: 76.7%
Validation accuracy: 74.0%
Loss at step 400: 1.458917
Training accuracy: 77.5%
Validation accuracy: 74.6%
Loss at step 500: 1.331289
Training accuracy: 78.1%
Validation accuracy: 74.8%
Loss at step 600: 1.231949
Training accuracy: 78.5%
Validation accuracy: 75.1%
Loss at step 700: 1.151889
Training accuracy: 79.2%
Validation accuracy: 75.3%
Loss at step 800: 1.085451
Training accuracy: 79.8%
Validation accuracy: 75.5%
Test accuracy: 82.7%


## Stochastic Gradient Descent (SGD)

Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a Placeholder node which will be fed actual data at every call of session.run().


In [6]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
    # now run the training
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 17.503656
Minibatch accuracy: 8.6%
Validation accuracy: 13.4%
Minibatch loss at step 500: 1.389075
Minibatch accuracy: 76.6%
Validation accuracy: 74.7%
Minibatch loss at step 1000: 0.825121
Minibatch accuracy: 79.7%
Validation accuracy: 76.7%
Minibatch loss at step 1500: 1.472916
Minibatch accuracy: 72.7%
Validation accuracy: 77.4%
Minibatch loss at step 2000: 1.342860
Minibatch accuracy: 75.0%
Validation accuracy: 77.8%
Minibatch loss at step 2500: 1.097952
Minibatch accuracy: 78.9%
Validation accuracy: 78.2%
Minibatch loss at step 3000: 1.113170
Minibatch accuracy: 75.8%
Validation accuracy: 78.2%
Test accuracy: 86.0%


## Comments
* SGD is much faster
* The accuracy for validationa dn test dataset has both improved with SGD
* Dont like the fact that the no of epochs(= num_steps) is very large and there is not visiblity of th epochs. 

Improvements could be provided by splititng the for loop in two parts, one for epoch another for total_batch; 
This way we could be sure that the whole training data is run through and randomization of bath data can also be done.

Other improvement could be by decreasing learning rate after each epoch.


## Problem

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units nn.relu() and 1024 hidden nodes. This model should improve your validation / test accuracy.

In [13]:
batch_size = 128
num_nodes_hidden_layer1 = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  #1st layer nodes = 784 =image_size*image_size
  #2nd layer nodes(hidden layer) = 1024(=num_nodes_hidden_layer1)
  #3rd layer nodes(output node, FC) = 10(= num_labels)
  # hiddenout = relu(X*W1 + b1); the shape of weights and input X should allow matrix multiplication
  # bias(b1), should be of shape [1, 1024]
  # X:shape = [batch_size,784]
  #define weights1 [784, 1024]
  weights1 = tf.Variable(tf.truncated_normal(shape=[image_size*image_size, num_nodes_hidden_layer1]))
  bias1 = tf.Variable(tf.zeros(shape =[1, num_nodes_hidden_layer1]))
  
  #hidden layer output = X*W1 +b1
  hidden_out = tf.add(tf.matmul(tf_train_dataset, weights1), bias1)
  hidden_out = tf.nn.relu(hidden_out)
  # shape of hidden_out = [batch_size, 1024]
  #now define the output layer as fully connected layer(FC)
  weights2 = tf.Variable(tf.truncated_normal(shape=[num_nodes_hidden_layer1, num_labels]))

  bias2 = tf.Variable(tf.zeros(shape=[1, num_labels]))
  prediction_logit = tf.add(tf.matmul(hidden_out, weights2), bias2)
  #prediction_out = tf.nn.softmax(prediction_logit)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= prediction_logit,labels= tf_train_labels))
  optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
  #predictions for train, valid and test data
  nntrain_prediction = tf.nn.softmax(prediction_logit)
  #define  afunction that calculates prediction using the layer1 and layer2  learned parameters
  def calc_prediction(X):
        layer1 = tf.nn.relu(tf.matmul(X, weights1)+ bias1)
        outputLayer = tf.nn.softmax(tf.matmul(layer1, weights2) + bias2)
        return outputLayer
  valid_prediction = calc_prediction(tf_valid_dataset)
  test_prediction = calc_prediction(tf_test_dataset)
    

In [16]:
# now run the training
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, nntrain_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Train Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 310.359222
Train Minibatch accuracy: 10.2%
Validation accuracy: 34.0%
Minibatch loss at step 500: 9.240896
Train Minibatch accuracy: 83.6%
Validation accuracy: 81.8%
Minibatch loss at step 1000: 6.662929
Train Minibatch accuracy: 85.9%
Validation accuracy: 83.2%
Minibatch loss at step 1500: 9.135612
Train Minibatch accuracy: 81.2%
Validation accuracy: 83.8%
Minibatch loss at step 2000: 10.140556
Train Minibatch accuracy: 79.7%
Validation accuracy: 83.2%
Minibatch loss at step 2500: 8.815826
Train Minibatch accuracy: 84.4%
Validation accuracy: 83.0%
Minibatch loss at step 3000: 7.356206
Train Minibatch accuracy: 81.2%
Validation accuracy: 83.8%
Test accuracy: 90.8%


### Discussion
I believe this NN model is prone to overfitting. The test dataset accuray is 90% for num_steps= 3001, and learning rate = 0.2;
However, if we increase num_steps = 4001, accracy in test dataset drops to 83%
Minibatch loss at step 4000: 5.965864
Train Minibatch accuracy: 77.3%
Validation accuracy: 77.5%
Test accuracy: 83.7%
### Clearly overfitting;
Next steps:
** include regularization in the model
** adopt a way to decrease the learning rate (alpha) over the epochs